## Rozwiązywanie układów równań liniowych metodami interacyjnymi

### Utills

In [340]:
import numpy as np
from random import randrange
import random

def get_random_vector(n):
    return np.random.uniform(low=-10.5, high=13.3, size=(n))


def get_random_matrix(n):
    A = np.random.uniform(low=-10.5, high=13.3, size=(n,n))
    A = A + A.T - np.diag(A.diagonal())
    for i in range(n):
        sum = 0
        for j in range(n):
            if i != j:
                sum += abs(A[i][j])
        A[i][i] = sum
    return A

### Zadanie 1
Zaimplementuj metodę Jacobiego. Podaj warunki jej stosowalności. Wygeneruj co najmniej trzy odpowiednie macierze o różnych wielkościach i sprawdź działanie swojej metody. Zwróć uwagę na zbieżność tej metody. 

Metoda Jacobiego spełnia warunki stosowalności, jeśli macierz A jest macierzą nieredukowalną oraz diagonalnie dominującą.

In [348]:
def jacobiego_method(A, B, N):
    x = np.zeros(len(A[0]))
    D = np.diag(A)
    R = A - np.diag(D)
    
    for i in range(N):
        new_x = np.dot(np.diag(1 / D), B - np.dot(R, x))
        if np.linalg.norm(new_x - x) < 1e-10:
            return new_x, i
        x = new_x
        
    return x, N


def test_jacobiego_method(n):
    A = get_random_matrix(n)
    B = get_random_vector(n)
   
    res, iterations = jacobiego_method(A, B, 1000)
    correct_res = np.linalg.solve(A, B)
    print('matrix size ', n)
    print("Correct result: ")
    print(correct_res)
    print("Jacobiego result:")
    print(res)
    print("Iterations number:", iterations)
    

for i in range(5, 15, 3):
    test_jacobiego_method(i)


LinAlgError: Singular matrix

Jak widać z testów algorytm działa poprawnie.

### Zadanie 2
Zaimplementuj metodę Gaussa-Seidla i kolejnych nadrelaksacji (successive over-relaxation). Podaj warunki stosowalności tych metod. Przeprowadź badanie działania swoich implementacji analogicznie jak w poprzednim zadaniu. Porównaj zbieżność wszystkich trzech metod.

Metoda Gaussa-Seidla spełnia warunki stosowalności, jeśli macierz A jest macierzą nieredukowalną oraz diagonalnie dominującą.

In [347]:
def gaussa_seidla_method(A, B, N):
    x = np.zeros(len(A[0]))
    
    for i in range(N):
        new_x = np.zeros(len(A[0]))
        for j in range(A.shape[0]):
            s1 = np.dot(A[j, :j], new_x[:j])
            s2 = np.dot(A[j, j + 1:], x[j + 1:])
            new_x[j] = (B[j] - s1 - s2) / A[j, j]
        if np.linalg.norm(new_x - x) < 1e-10:
            return new_x, i
        x = new_x
    return x, N


def test_gaussa_seidla_method(n):
    A = get_random_matrix(n)
    B = get_random_vector(n)
   
    res, iterations = gaussa_seidla_method(A, B, 1000)
    correct_res = np.linalg.solve(A, B)
    print('matrix size ', n)
    print("Correct result: ")
    print(correct_res)
    print("Gaussa-Seidla result:")
    print(res)
    print("Iterations number:", iterations)
    

for i in range(5, 15, 3):
    test_gaussa_seidla_method(i)

('matrix size ', 5)
Correct result: 
[ 0.06007411 -0.01140587 -0.13886013  0.01241818  0.27861815]
Gaussa-Seidla result:
[ 0.06007411 -0.01140587 -0.13886013  0.01241818  0.27861815]
('Iterations number:', 54)
('matrix size ', 8)
Correct result: 
[-0.12027399  0.10988583  0.17651817 -0.14420489 -0.07091756 -0.13888164
  0.00194861 -0.11453759]
Gaussa-Seidla result:
[-0.12027399  0.10988583  0.17651817 -0.14420489 -0.07091756 -0.13888164
  0.00194861 -0.11453759]
('Iterations number:', 20)
('matrix size ', 11)
Correct result: 
[ 0.08496573 -0.00120639 -0.17136677  0.15087364  0.20402514  0.15829568
  0.0316235  -0.0992635  -0.00341183 -0.22717849  0.14841047]
Gaussa-Seidla result:
[ 0.08496573 -0.00120639 -0.17136677  0.15087364  0.20402514  0.15829568
  0.0316235  -0.0992635  -0.00341183 -0.22717849  0.14841047]
('Iterations number:', 30)
('matrix size ', 14)
Correct result: 
[ 0.03545921  0.13740671  0.02996968 -0.12493287  0.14236997  0.004671
  0.11841591  0.00591958  0.13693188  0.

Jak widać z testów algorytm działa poprawnie.

Metoda successive over-relaxation spełnia warunki stosowalności, jeśli macierz A jest macierzą nieredukowalną oraz diagonalnie dominującą.

In [346]:
def sor_method(A, B, omega, N):
    x = np.zeros(len(A[0]))
    new_x = np.zeros(len(A[0]))

    for i in range(N):
        for j in range (B.shape[0]):
            x[j] = (B[j] - (np.dot(A[j, j + 1:], new_x[j + 1:]) + 
                            np.dot(A[j, :j], x[:j]))) / A[j, j]
            x[j] = np.dot(x[j], omega) + np.dot(new_x[j], (1 - omega))

        if np.linalg.norm(np.dot(A, x) - B) < 1e-10:
            return new_x, j
        new_x = x
    return x, N


def test_sor_method(n):
    A = get_random_matrix(n)
    B = get_random_vector(n)
   
    res, iterations = sor_method(A, B, 0.5, 1000)
    correct_res = np.linalg.solve(A, B)
    print('matrix size ', n)
    print("Correct result: ")
    print(correct_res)
    print("SOR result:")
    print(res)
    print("Iterations number:", iterations)
    

for i in range(5, 15, 3):
    test_sor_method(i)

('matrix size ', 5)
Correct result: 
[ 0.59440904 -0.51118647 -0.72106637  0.42229232 -0.0839743 ]
SOR result:
[ 0.59440904 -0.51118647 -0.72106637  0.42229232 -0.0839743 ]
('Iterations number:', 4)
('matrix size ', 8)
Correct result: 
[ 0.07662817  0.16062039  0.17864795 -0.07682647  0.17060026  0.17286401
  0.01012103  0.09185987]
SOR result:
[ 0.07662817  0.16062039  0.17864795 -0.07682647  0.17060026  0.17286401
  0.01012103  0.09185987]
('Iterations number:', 7)
('matrix size ', 11)
Correct result: 
[ 0.08699728 -0.0930432  -0.05891878 -0.1136331  -0.18287995  0.13923411
 -0.10672368 -0.03917497 -0.06771489  0.03972803  0.03147531]
SOR result:
[ 0.08699728 -0.0930432  -0.05891878 -0.1136331  -0.18287995  0.13923411
 -0.10672368 -0.03917497 -0.06771489  0.03972803  0.03147531]
('Iterations number:', 10)
('matrix size ', 14)
Correct result: 
[ 0.00405041 -0.01419479 -0.0834915   0.13950805  0.05011546 -0.07449952
  0.02751264 -0.07726196  0.01401379  0.1415579   0.02850807 -0.004016

Jak widać z testów algorytm działa poprawnie.

In [345]:
def compare_iterations(n):
    A = get_random_matrix(n)
    B = get_random_vector(n)
    
    _, iterations1 = jacobiego_method(A, B, 1000)
    _, iterations2 = gaussa_seidla_method(A, B, 1000)
    _, iterations3 = sor_method(A, B, 0.5, 1000)
    
    print("Jacobiego iterations number:", iterations1)
    print("Gaussa-Seidla iterations number:", iterations2)
    print("SOR iterations number:", iterations3)
    
compare_iterations(10)

('Jacobiego iterations number:', 62)
('Gaussa-Seidla iterations number:', 24)
('SOR iterations number:', 9)


Metoda SOR ma najmniej iteracji, na drugim miejscu metoda Gaussa-Seidla, na trzecim metoda Jacobiego.